In [ ]:
import numpy as np 
import pandas as pd
import matplotlib as plt
from IPython.display import display
import talib as ta


def adjust_data(dataname):
    df = pd.read_csv(dataname, index_col='Datetime')

    
    #以降全ての計算でレート終値を使う
    close = np.array(df["Close"])
    
    
    #特徴量を入れるための空のdataframeを作成
    df_feature = pd.DataFrame(columns=[ 
        "January","February","March","April","May","June",
        "July","August","September","October","November","December",
        "day",
        "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
        "SMA_hour/current",
        "SMA_2hour/current",
        "RSI",
        "MACD",
        "BBANDS+2σ",
        "BBANDS-2σ",
        "10m_rate",
        "30m_rate",
        "60m_rate",
        "120m_rate"
        ])


    df_feature["January"] = df["January"]
    df_feature["February"] = df["February"]
    df_feature["March"] = df["March"]
    df_feature["April"] = df["April"]
    df_feature["May"] = df["May"]
    df_feature["June"] = df["June"]
    df_feature["July"] = df["July"]
    df_feature["August"] = df["August"]
    df_feature["September"] = df["September"]
    df_feature["October"] = df["October"]
    df_feature["November"] = df["November"]
    df_feature["December"] = df["December"]
    
    #print(df_feature.isnull().any())
    
    #print(df_feature.isnull().any())
    
#     days=["Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday"]
#     for i in days:
#         df_feature[i] = df[i]

    df_feature["Sunday"] = df["Sunday"]
    df_feature["Monday"] = df["Monday"]
    df_feature["Thuesday"] = df["Thuesday"]
    df_feature["Wednesday"] = df["Wednesday"]
    df_feature["Thursday"] = df["Thursday"]
    df_feature["Friday"] = df["Friday"]
    df_feature["Saturday"] = df["Saturday"]
    
    
    
    df_feature["day"] = df["day"]
    df_feature["10m_rate"] = df["Close"].pct_change()*100
    df_feature["30m_rate"] = df["Close"].pct_change(3)*100
    df_feature["60m_rate"] = df["Close"].pct_change(6)*100
    df_feature["120m_rate"] = df["Close"].pct_change(12)*100

    
    
    
    #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる

    #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df_feature["SMA_hour/current"]= ta.SMA(close, timeperiod=6) / close
    df_feature["SMA_2hour/current"]= ta.SMA(close, timeperiod=12) / close

    #RSI
    df_feature["RSI"] = ta.RSI(close, timeperiod=12)

    #MACD
    df_feature["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)

    #ボリンジャーバンド 
    upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=3, nbdevdn=3)
    df_feature["BBANDS+2σ"] = upper / close
    df_feature["BBANDS-2σ"] = lower / close

    
    
    c=[0,0,0]
    
    
    def classify(x):
        percent = 0.1

        if x<-percent:
            c[0]+=1
            return 0
        elif -percent<x<percent:
            c[1]+=1
            return 1
        elif percent<x:
            c[2]+=1
            return 2
       

    classified = pd.DataFrame(columns=["class"])
    classified["class"] = df_feature["120m_rate"].apply(lambda x: classify(x))

    classified = classified.shift(-1)
    
    
    
    df_adjusted=df_feature[30:len(df_feature)-2]
    classified=classified[30:len(classified)-2]
    
    print(c)
    
    return df_adjusted, classified


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import optuna


df_adjusted, classified = adjust_data("./USD_JPY_201601-201908_M10.csv")

X_train, X_test, y_train, y_test = train_test_split(df_adjusted, classified["class"], train_size=0.8)


#y_ravel=np.ravel(y_train)

#print(X_train, y_train)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print(accuracy_score(y_test, clf.predict(X_test)))

#X_real, y_real = adjust_data("./USD_JPY_202009_M10.csv")
#print(accuracy_score(y_real, clf.predict(X_real)))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, clf.predict(X_test)))

import pickle
filename = 'RF_01.sav'
pickle.dump(clf, open(filename, 'wb'))

# print(X_train)
#print(clf.feature_importances_)
    